In [ ]:
import sys
sys.path.append("../../")
from llm_api import ClientLLM
from utils import get_tui_columns, load_samples, load_promts
import time

In [ ]:
api_key = "FALSE" 
api_base = "http://localhost:8888/v1"
model = "meta-llama/Llama-2-7b-chat-hf" 
temperature = 0.5


chat_instance = ClientLLM(model, api_key, api_base, temperature)

In [ ]:
csv_file_path = '../../promts/tuis_desc.csv'
tuis = get_tui_columns(csv_file_path, columns='name')
print(tuis)

In [ ]:
disease = "Mycosis"
text = "Most common mild mycoses often present with a rash. Infections within the skin or under the skin may present with a lump and skin changes. Less common deeper fungal infections may present with pneumonia like symptoms or meningitis."

In [ ]:
gold = "rash;lump; skin changes; pneumonia;meningitis."

In [ ]:
prompts = load_promts("../../promts/prompts.xlsx")
prompts

In [ ]:
prompt = prompts[3]
prompt

In [ ]:
prompt_content = prompt.msg.format(disease=disease, tuis=tuis, text=text)
template = [{"role": "user", "content": prompt_content}]
template

In [ ]:
response = chat_instance.ask_complete(template, 100)

In [ ]:
response.usage.prompt_tokens

In [ ]:
response.choices[0].message.content

## Model

In [ ]:
from tqdm import tqdm
import json
from time import sleep

In [ ]:
PATH = "../../standard/golds_v4.json"

In [ ]:
golds = []

with open(PATH) as f:
    golds = json.load(f)

# No External Knowledge

## Zero-Shot

In [ ]:
tokens = {}

In [ ]:
results_zero = []
tokens["zero"] = {"base": prompt.n_tokens ,"total" : 0}

for ds, val in tqdm(golds.items()):
    prompt_content = prompt.msg.format(disease=ds, tuis=tuis, text=val["text"])
    promt = [{"role": "user", "content": prompt_content}]
    response = chat_instance.ask_complete(promt, 100)
    answ = response.choices[0].message.content
    sleep(2)
    results_zero.append({"y_pred":answ, "y_true":val["sings"]})
    tokens["zero"]["total"] += response.usage.total_tokens

In [ ]:
results_zero[-1]

In [ ]:
with open("../../results/llama7b_zero_results.json", "w") as f:
    json.dump(results_zero, f, indent=2)

In [ ]:
tokens

# Few-Shot

In [ ]:
samples = '../../promts/samples.csv'
texts, symptoms_lists = load_samples(samples)

## 1-shot

In [ ]:
results_1_shot = []
tokens["1-shot"] = {"base": prompt.n_tokens ,"total" : 0}

for ds, val in tqdm(golds.items()):
    prompt_content = prompt.msg.format(disease=ds, tuis=tuis, text=val["text"])
    promt =  [{"role": "user", "content": texts[0]},{"role": "assistant", "content": symptoms_lists[0]},{"role": "user", "content": prompt_content}]
    response = chat_instance.ask_complete(promt, 100)
    time.sleep(2)
    answ = response.choices[0].message.content
    results_1_shot.append({"y_pred":answ, "y_true":val["sings"]})
    tokens["1-shot"]["total"] += response.usage.total_tokens

In [ ]:
results_1_shot[:2]

In [ ]:
with open("../../results/llama7b_1_shot_results.json", "w") as f:
    json.dump(results_1_shot, f, indent=2)

In [ ]:
tokens

## 3-shot

In [ ]:
results_3_shot = []
tokens["3-shot"] = {"base": prompt.n_tokens ,"total" : 0}

for ds, val in tqdm(golds.items()):
    prompt_content = prompt.msg.format(disease=ds, tuis=tuis, text=val["text"])
    promt =  [{"role": "user", "content": texts[0]},{"role": "assistant", "content": symptoms_lists[0]}, {"role": "user", "content": texts[1]},{"role": "assistant", "content": symptoms_lists[1]}, {"role": "user", "content": texts[2]},{"role": "assistant", "content": symptoms_lists[2]},{"role": "user", "content": prompt_content}]
    response = chat_instance.ask_complete(promt, 100)
    time.sleep(2)
    answ = response.choices[0].message.content
    results_3_shot.append({"y_pred":answ, "y_true":val["sings"]})
    tokens["3-shot"]["total"] += response.usage.total_tokens

In [ ]:
results_3_shot[:2]

In [ ]:
with open("../../results/llama7b_3_shot_results.json", "w") as f:
    json.dump(results_3_shot, f, indent=2)

## 5-shot

In [ ]:
results_5_shot = []
tokens["5-shot"] = {"base": prompt.n_tokens ,"total" : 0}

for ds, val in tqdm(golds.items()):
    prompt_content = prompt.msg.format(disease=ds, tuis=tuis, text=val["text"])
    promt =  [{"role": "user", "content": texts[0]},{"role": "assistant", "content": symptoms_lists[0]}, {"role": "user", "content": texts[1]},{"role": "assistant", "content": symptoms_lists[1]}, {"role": "user", "content": texts[2]},{"role": "assistant", "content": symptoms_lists[2]},{"role": "user", "content": texts[3]},{"role": "assistant", "content": symptoms_lists[3]},{"role": "user", "content": texts[4]},{"role": "assistant", "content": symptoms_lists[4]},{"role": "user", "content": prompt_content}]
    response = chat_instance.ask_complete(promt, 100)
    time.sleep(2)
    answ = response.choices[0].message.content
    results_5_shot.append({"y_pred":answ, "y_true":val["sings"]})
    tokens["5-shot"]["total"] += response.usage.total_tokens

In [ ]:
results_5_shot[:2]

In [ ]:
with open("../../results/llama7b_5_shot_results.json", "w") as f:
    json.dump(results_5_shot, f, indent=2)

## 10-shot

In [ ]:
results_10_shot = []
tokens["10-shot"] = {"base": prompt.n_tokens ,"total" : 0}

for ds, val in tqdm(golds.items()):
    prompt_content = prompt.msg.format(disease=ds, tuis=tuis, text=val["text"])
    promt =  [{"role": "user", "content": texts[0]},{"role": "assistant", "content": symptoms_lists[0]}, {"role": "user", "content": texts[1]},{"role": "assistant", "content": symptoms_lists[1]}, {"role": "user", "content": texts[2]},{"role": "assistant", "content": symptoms_lists[2]},{"role": "user", "content": texts[3]},{"role": "assistant", "content": symptoms_lists[3]},{"role": "user", "content": texts[4]},{"role": "assistant", "content": symptoms_lists[4]},{"role": "user", "content": texts[5]},{"role": "assistant", "content": symptoms_lists[5]},{"role": "user", "content": texts[6]},{"role": "assistant", "content": symptoms_lists[6]},{"role": "user", "content": texts[7]},{"role": "assistant", "content": symptoms_lists[7]},{"role": "user", "content": texts[8]},{"role": "assistant", "content": symptoms_lists[8]},{"role": "user", "content": texts[9]},{"role": "assistant", "content": symptoms_lists[9]},{"role": "user", "content": prompt_content}]
    response = chat_instance.ask_complete(promt, 100)
    time.sleep(2)
    answ = response.choices[0].message.content
    results_10_shot.append({"y_pred":answ, "y_true":val["sings"]})
    tokens["10-shot"]["total"] += response.usage.total_tokens

In [ ]:
results_10_shot[:2]

In [ ]:
with open("../../results/llama7b_10_shot_results.json", "w") as f:
    json.dump(results_10_shot, f, indent=2)

# External Knowledge

In [ ]:
tuis = get_tui_columns(csv_file_path, columns='both')
tuis

## Zero-Shot

In [ ]:
results_zero_e = []
tokens["zero_e"] = {"base": prompt.n_tokens ,"total" : 0}

for ds, val in tqdm(golds.items()):
    prompt_content = prompt.msg.format(disease=ds, tuis=tuis, text=val["text"])
    promt = [{"role": "user", "content": prompt_content}]
    response = chat_instance.ask_complete(promt, 100)
    answ = response.choices[0].message.content
    sleep(2)
    results_zero_e.append({"y_pred":answ, "y_true":val["sings"]})
    tokens["zero_e"]["total"] += response.usage.total_tokens

In [ ]:
results_zero_e[:2]

In [ ]:
with open("../../results/llama7b_zero_e_results.json", "w") as f:
    json.dump(results_zero_e, f, indent=2)

# Few-Shot

## 1-shot

In [ ]:
results_1_shot_e = []
tokens["1_shot_e"] = {"base": prompt.n_tokens ,"total" : 0}

for ds, val in tqdm(golds.items()):
    prompt_content = prompt.msg.format(disease=ds, tuis=tuis, text=val["text"])
    promt =  [{"role": "user", "content": texts[0]},{"role": "assistant", "content": symptoms_lists[0]},{"role": "user", "content": prompt_content}]
    response = chat_instance.ask_complete(promt, 100)
    time.sleep(2)
    answ = response.choices[0].message.content
    results_1_shot_e.append({"y_pred":answ, "y_true":val["sings"]})
    tokens["1_shot_e"]["total"] += response.usage.total_tokens

In [ ]:
results_1_shot_e[-2]

In [ ]:
with open("../../results/llama7b_1_shot_e_results.json", "w") as f:
    json.dump(results_1_shot_e, f, indent=2)

## 3-shot

In [ ]:
results_3_shot_e = []
tokens["3_shot_e"] = {"base": prompt.n_tokens ,"total" : 0}

for ds, val in tqdm(golds.items()):
    prompt_content = prompt.msg.format(disease=ds, tuis=tuis, text=val["text"])
    promt =  [{"role": "user", "content": texts[0]},{"role": "assistant", "content": symptoms_lists[0]}, {"role": "user", "content": texts[1]},{"role": "assistant", "content": symptoms_lists[1]}, {"role": "user", "content": texts[2]},{"role": "assistant", "content": symptoms_lists[2]},{"role": "user", "content": prompt_content}]
    response = chat_instance.ask_complete(promt, 100)
    time.sleep(2)
    answ = response.choices[0].message.content
    results_3_shot_e.append({"y_pred":answ, "y_true":val["sings"]})
    tokens["3_shot_e"]["total"] += response.usage.total_tokens

In [ ]:
results_3_shot_e[:2]

In [ ]:
with open("../../results/llama7b_3_shot_e_results.json", "w") as f:
    json.dump(results_3_shot_e, f, indent=2)

## 5-shot

In [ ]:
results_5_shot_e = []
tokens["5_shot_e"] = {"base": prompt.n_tokens ,"total" : 0}

for ds, val in tqdm(golds.items()):
    prompt_content = prompt.msg.format(disease=ds, tuis=tuis, text=val["text"])
    promt =  [{"role": "user", "content": texts[0]},{"role": "assistant", "content": symptoms_lists[0]}, {"role": "user", "content": texts[1]},{"role": "assistant", "content": symptoms_lists[1]}, {"role": "user", "content": texts[2]},{"role": "assistant", "content": symptoms_lists[2]},{"role": "user", "content": texts[3]},{"role": "assistant", "content": symptoms_lists[3]},{"role": "user", "content": texts[4]},{"role": "assistant", "content": symptoms_lists[4]},{"role": "user", "content": prompt_content}]
    response = chat_instance.ask_complete(promt, 100)
    time.sleep(2)
    answ = response.choices[0].message.content
    results_5_shot_e.append({"y_pred":answ, "y_true":val["sings"]})
    tokens["5_shot_e"]["total"] += response.usage.total_tokens

In [ ]:
results_5_shot_e[:2]

In [ ]:
with open("../../results/llama7b_5_shot_e_results.json", "w") as f:
    json.dump(results_5_shot_e, f, indent=2)

## 10-shot

In [ ]:
results_10_shot_e = []
tokens["10_shot_e"] = {"base": prompt.n_tokens ,"total" : 0}

for ds, val in tqdm(golds.items()):
    prompt_content = prompt.msg.format(disease=ds, tuis=tuis, text=val["text"])
    promt =  [{"role": "user", "content": texts[0]},{"role": "assistant", "content": symptoms_lists[0]}, {"role": "user", "content": texts[1]},{"role": "assistant", "content": symptoms_lists[1]}, {"role": "user", "content": texts[2]},{"role": "assistant", "content": symptoms_lists[2]},{"role": "user", "content": texts[3]},{"role": "assistant", "content": symptoms_lists[3]},{"role": "user", "content": texts[4]},{"role": "assistant", "content": symptoms_lists[4]},{"role": "user", "content": texts[5]},{"role": "assistant", "content": symptoms_lists[5]},{"role": "user", "content": texts[6]},{"role": "assistant", "content": symptoms_lists[6]},{"role": "user", "content": texts[7]},{"role": "assistant", "content": symptoms_lists[7]},{"role": "user", "content": texts[8]},{"role": "assistant", "content": symptoms_lists[8]},{"role": "user", "content": texts[9]},{"role": "assistant", "content": symptoms_lists[9]},{"role": "user", "content": prompt_content}]
    response = chat_instance.ask_complete(promt, 100)
    time.sleep(2)
    answ = response.choices[0].message.content
    results_10_shot_e.append({"y_pred":answ, "y_true":val["sings"]})
    tokens["10_shot_e"]["total"] += response.usage.total_tokens

In [ ]:
results_10_shot_e[:2]

In [ ]:
with open("../../results/llama7b_10_shot_e_results.json", "w") as f:
    json.dump(results_10_shot_e, f, indent=2)